In [1]:
!pip install firebase-admin
import firebase_admin
from firebase_admin import credentials, db
from datetime import datetime  # Untuk konversi timestamp ke human-readable
import pandas as pd


[notice] A new release of pip is available: 23.0.1 -> 25.2
[notice] To update, run: pip install --upgrade pip


In [2]:
# Konfigurasi database sumber
source_cred = credentials.Certificate("/work/staklimjerukagung-firebase-admin.json")
source_app = firebase_admin.initialize_app(source_cred, {
  'databaseURL': 'https://staklimjerukagung-default-rtdb.asia-southeast1.firebasedatabase.app/'
})

In [3]:
from datetime import datetime
from zoneinfo import ZoneInfo

# Input tanggal dan waktu dalam format string
start_readable_date = "07-09-2025 18:53:00"
end_readable_date = "07-09-2025 20:03:52"

# 1. Tentukan zona waktu yang diinginkan
jakarta_tz = ZoneInfo("Asia/Jakarta")

# 2. Ubah string menjadi objek datetime yang "naive" (tanpa timezone)
naive_start_dt = datetime.strptime(start_readable_date, "%d-%m-%Y %H:%M:%S")
naive_end_dt = datetime.strptime(end_readable_date, "%d-%m-%Y %H:%M:%S")

# 3. Jadikan objek datetime "aware" dengan melokalkannya ke zona waktu Jakarta
aware_start_dt = naive_start_dt.replace(tzinfo=jakarta_tz)
aware_end_dt = naive_end_dt.replace(tzinfo=jakarta_tz)

# 4. Sekarang konversi ke Unix timestamp. Hasilnya akan akurat berdasarkan WIB.
start_timestamp = int(aware_start_dt.timestamp())
end_timestamp = int(aware_end_dt.timestamp())

# --- Verifikasi ---
print(f"String Asli       : {start_readable_date}")
print(f"Waktu (Aware WIB)  : {aware_start_dt}")
print(f"Unix Timestamp     : {start_timestamp}")
print("-" * 30)
print(f"String Asli       : {end_readable_date}")
print(f"Waktu (Aware WIB)  : {aware_end_dt}")
print(f"Unix Timestamp     : {end_timestamp}")

String Asli       : 07-09-2025 18:53:00
Waktu (Aware WIB)  : 2025-09-07 18:53:00+07:00
Unix Timestamp     : 1757245980
------------------------------
String Asli       : 07-09-2025 20:03:52
Waktu (Aware WIB)  : 2025-09-07 20:03:52+07:00
Unix Timestamp     : 1757250232


In [4]:
# Referensi ke data sumber
source_ref = db.reference('/auto_weather_stat/id-04/data')

# Mengambil data dari database sumber
source_data = source_ref.get()

# Mengubah data menjadi DataFrame pandas
if source_data:
    # Mengubah data menjadi DataFrame
    df = pd.DataFrame.from_dict(source_data, orient='index')
    
    # Jika 'timestamp' sudah ada, jangan pindahkan indeks ke kolom
    if 'timestamp' not in df.columns:
        df.index.name = 'timestamp'  # Mengatur nama indeks
        df.reset_index(inplace=True)  # Memindahkan indeks menjadi kolom biasa
    
    print(df.head())  # Menampilkan data untuk verifikasi
else:
    print("Tidak ada data di databse sumber.")

                 dew  humidity  pressure  temperature   timestamp     volt
1743465613  27.84270     88.54   1009.52        29.95  1743465613  4.21750
1743465628  27.86640     88.51   1009.52        29.98  1743465628  4.21750
1743465643  27.85448     88.55   1009.52        29.96  1743465643  4.21750
1743465658  27.88387     88.60   1009.51        29.98  1743465658  4.21625
1743465673  27.86418     88.60   1009.52        29.96  1743465673  4.21625


In [5]:
df.head(10)

,dew,humidity,pressure,temperature,timestamp,volt
1743465613,27.84270,88.54,1009.52,29.95,1743465613,4.21750
1743465628,27.86640,88.51,1009.52,29.98,1743465628,4.21750
1743465643,27.85448,88.55,1009.52,29.96,1743465643,4.21750
1743465658,27.88387,88.60,1009.51,29.98,1743465658,4.21625
1743465673,27.86418,88.60,1009.52,29.96,1743465673,4.21625
1743465688,27.88774,88.62,1009.52,29.98,1743465688,4.21625
1743465703,27.89371,88.60,1009.53,29.99,1743465703,4.21750
1743465718,27.86418,88.60,1009.53,29.96,1743465718,4.21750
1743465733,27.88387,88.60,1009.53,29.98,1743465733,4.21750
1743465748,27.88774,88.62,1009.53,29.98,1743465748,4.21625


In [6]:
# Filter data berdasarkan rentang waktu dengan validasi kunci
filtered_data = {
    key: value
    for key, value in source_data.items()
    if key.isdigit() and start_timestamp <= int(key) <= end_timestamp
}

In [7]:
# Konfigurasi database tujuan
dest_cred = credentials.Certificate("/work/staklimjerukagung-firebase-admin.json")
dest_app = firebase_admin.initialize_app(dest_cred, {
    'databaseURL': 'https://staklimjerukagung-default-rtdb.asia-southeast1.firebasedatabase.app/'
}, name='destination')

In [8]:
# Referensi ke data tujuan
dest_ref = db.reference('/auto_weather_stat/id-05/data', app=dest_app)

In [9]:
# Memindahkan data yang telah difilter ke database tujuan
if filtered_data:
    dest_ref.update(filtered_data)
    print("Data berhasil dipindahkan ke database tujuan.")
else:
    print("Tidak ada data dalam rentang waktu yang ditentukan.")

Data berhasil dipindahkan ke database tujuan.


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=200afad5-8991-4e20-8a55-8751c7aff3b5' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>